In [ ]:
!pip freeze

In [ ]:
!python -V

In [1]:
%%time
# imports
import os
import time
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers, models, Input
import tensorflow as tf

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

2024-12-06 19:32:44.488688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733531565.514997   16062 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733531565.782370   16062 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 19:32:48.384532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
CPU times: user 5.08 s, sys: 1.98 s, total: 7.07 s
Wall time: 58.1 s


In [ ]:
# %%time
# # read in data and preprocess
# images_path = './data/archive3/ASL_Alphabet_Dataset/asl_alphabet_train/'
# images = []

# for directory in os.listdir(images_path):
#     count = 0
#     for file in os.listdir(images_path+directory):
#         file_path = images_path+directory+'/'+file
#         image = Image.open(file_path)
#         image = image.convert('L') # convert to grayscale
#         image = image.resize((128, 128)) # resize
#         # image.save('./data/resized/'+file) # uncomment to save the resized image
#         image = np.array(image) # convert to array
#         image = image / 255.0 # normalize
#         image = image.astype(np.float32)
#         if count % 1000 == 0:
#             images.append((directory, image))
#         count = count + 1

# x = [image[1] for image in images]
# y = [image[0] for image in images]

In [ ]:
# %%time
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# x_train = np.array(x_train)
# x_test = np.array(x_test)


# # binarize the labels
# label_binarizer = LabelBinarizer()
# y_train = label_binarizer.fit_transform(y_train)
# y_test = label_binarizer.transform(y_test)

# print(x_train.shape)
# print(y_train.shape)
# print(x_test.shape)
# print(y_test.shape)

In [ ]:
# x_train_memmap = np.memmap('x_train_memmap.dat', dtype=x_train.dtype, mode='w+', shape=x_train.shape)
# y_train_memmap = np.memmap('y_train_memmap.dat', dtype=y_train.dtype, mode='w+', shape=y_train.shape)
# x_test_memmap = np.memmap('x_test_memmap.dat', dtype=x_test.dtype, mode='w+', shape=x_test.shape)
# y_test_memmap = np.memmap('y_test_memmap.dat', dtype=y_test.dtype, mode='w+', shape=y_test.shape)

# x_train_memmap[:] = x_train
# y_train_memmap[:] = y_train
# x_test_memmap[:] = x_test
# y_test_memmap[:] = y_test

# x_train_memmap.flush()
# y_train_memmap.flush()
# x_test_memmap.flush()
# y_test_memmap.flush()

In [ ]:
# x_train = np.memmap('x_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 128, 128))
# y_train = np.memmap('y_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 29))
# x_test = np.memmap('x_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 128, 128))
# y_test = np.memmap('y_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 29))

# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [8]:
# x_train = np.memmap('x_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 128, 128))
# y_train = np.memmap('y_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 29))
# x_test = np.memmap('x_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 128, 128))
# y_test = np.memmap('y_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 29))
# 
# rat = 20
# x_train = x_train[:len(x_train)//rat]
# y_train = y_train[:len(y_train)//rat]
# x_test = x_test[:len(x_test)//rat]
# y_test = y_test[:len(y_test)//rat]
# print(len(x_train))
# print(len(x_test))

In [2]:
x_train = np.memmap('x_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 128, 128))
y_train = np.memmap('y_train_memmap.dat', dtype=np.float32, mode='r', shape=(178459, 29))
x_test = np.memmap('x_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 128, 128))
y_test = np.memmap('y_test_memmap.dat', dtype=np.float32, mode='r', shape=(44615, 29))

def memmap_generator(x_memmap, y_memmap):
    for i in range(len(x_memmap)):
        yield x_memmap[i], y_memmap[i]

train_dataset = tf.data.Dataset.from_generator(
    lambda: memmap_generator(x_train, y_train),
    output_signature=(
        tf.TensorSpec(shape=(128, 128), dtype=tf.float32),
        tf.TensorSpec(shape=(29,), dtype=tf.float32)
    )
)

test_dataset = tf.data.Dataset.from_generator(
    lambda: memmap_generator(x_test, y_test),
    output_signature=(
        tf.TensorSpec(shape=(128, 128), dtype=tf.float32),
        tf.TensorSpec(shape=(29,), dtype=tf.float32)
    )
)

train_dataset = (
    train_dataset
    .map(lambda x, y: (tf.expand_dims(x, axis=-1), y))
    .shuffle(buffer_size=1000)
    .batch(128)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_dataset = (
    test_dataset
    .map(lambda x, y: (tf.expand_dims(x, axis=-1), y))
    .batch(128)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

train_dataset = train_dataset.map(lambda x, y: (x, tf.cast(y, tf.float32)))
test_dataset = test_dataset.map(lambda x, y: (x, tf.cast(y, tf.float32)))




I0000 00:00:1733531604.773513   16062 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:02:00.0, compute capability: 8.6
I0000 00:00:1733531604.778188   16062 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6692 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# tf.debugging.set_log_device_placement(True)

In [3]:
%%time

# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
# with strategy.scope():
model = models.Sequential([
    layers.Conv2D(10, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(29, activation='softmax')
])
            
model.compile(optimizer='adam',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# for layer in model.layers:
#     print(layer.name, [w.dtype for w in layer.weights])

/mnt/c/Users/John Califano/Desktop/AIProj/live_ASL_reader/wsl/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


CPU times: user 469 ms, sys: 207 ms, total: 677 ms
Wall time: 2.51 s


In [ ]:
%%time
# with tf.device('/GPU:0'):
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

In [ ]:
%%time
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=test_dataset
)

Epoch 1/30


I0000 00:00:1733531619.534842   16201 service.cc:148] XLA service 0x7fc1f400c0b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733531619.537555   16201 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
I0000 00:00:1733531619.537588   16201 service.cc:156]   StreamExecutor device (1): NVIDIA GeForce GTX 1070 Ti, Compute Capability 6.1
2024-12-06 19:33:39.772615: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733531620.130800   16201 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-06 19:33:41.006938: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_658', 16 bytes spill stores, 16 bytes spill loads



      1/Unknown 11s 11s/step - accuracy: 0.0312 - loss: 2.8026e-45

I0000 00:00:1733531627.419946   16201 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   1395/Unknown 482s 338ms/step - accuracy: 0.0374 - loss: 3.4487e-41

...

2024-12-06 23:30:21.432765: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11981823494394289211
2024-12-06 23:30:21.432824: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6425893941402143907


1395/1395 ━━━━━━━━━━━━━━━━━━━━ 613s 437ms/step - accuracy: 0.0355 - loss: 2.8957e-39 - val_accuracy: 0.0149 - val_loss: 0.0000e+00
Epoch 25/30


2024-12-06 23:32:22.383843: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11981823494394289211
2024-12-06 23:32:22.383903: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6425893941402143907


1392/1395 ━━━━━━━━━━━━━━━━━━━━ 1s 356ms/step - accuracy: 0.0355 - loss: 3.0360e-39  

2024-12-06 23:40:40.921720: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11981823494394289211
2024-12-06 23:40:40.921806: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 6425893941402143907


# Performance Graph

In [ ]:
# plot model performance
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

# Loss and Accuracy measures

In [ ]:
# print results of test data ran through model
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(test_loss)
print(test_accuracy)

In [ ]:
classes = { 0: "A",
            1: "B",
            2: "C",
            3: "D",
            4: "del",
            5: "E",
            6: "F",
            7: "G",
            8: "H",
            9: "I",
            10: "J",
            11: "K",
            12: "L",
            13: "M",
            14: "N",
            15: "nothing",
            16: "O",
            17: "P",
            18: "Q",
            19: "R",
            20: "S",
            21: "space",
            22: "T",
            23: "U",
            24: "V",
            25: "W",
            26: "X",
            27: "Y",
            28: "Z",
            }

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12, 12))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes.values())
disp.plot( values_format='d', ax=plt.gca())
plt.xticks(rotation=90)
plt.show()

# Classification Report

In [ ]:
 print(classification_report(y_true, y_pred_classes, target_names= classes.values())) 

## If model seems good, save it with the following cell

In [ ]:
# save model and lebal binarizer with date and time
from datetime import datetime
keras_model_save_name = './keras_saves/' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.keras'
label_binarizer_save_name = './binarizer/' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl'
joblib.dump(label_binarizer, label_binarizer_save_name) # save label binarizer
model.save(keras_model_save_name) # save keras model

## The following cells are not needed for the research portion. They will only be used when we decide what model is best after research

In [ ]:
model_name_to_export = 'NewExample'

In [ ]:
# load trained model
from tensorflow.keras.models import load_model
load_name ='./keras_saves/' +  model_name_to_export + '.keras'
model = load_model(load_name, compile=False) 
print(model.summary())

In [ ]:
# export model
model.export('./model_exports/')

In [ ]:
# load exported model
import tensorflow as tf
reloaded_artifact = tf.saved_model.load('./model_exports/')
print(reloaded_artifact.signatures)


In [ ]:
# prepocess test image
# test_img_path = './data/archive/asl_alphabet_test/asl_alphabet_test/L_test.jpg'
test_img_path = './data/test/test4.jpeg'
og_image = Image.open(test_img_path)
image = og_image.convert('L') # convert to grayscale
image = image.resize((128, 128)) # resize
image = np.array(image) # convert to array
image = image / 255.0 # normalize
image_array = np.expand_dims(image, axis=-1) 
image_array = np.expand_dims(image_array, axis=0) 


In [ ]:
# predict test image
predictions = reloaded_artifact.serve(image_array)
rounded = np.round(predictions)
b_load_name = './binarizer/' + model_name_to_export + '.pkl'
label_binarizer = joblib.load(b_load_name) 
original_label = label_binarizer.inverse_transform(rounded)
print(original_label)